# The Spinning Effective One-Body Initial Condition Solver

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body initial condition solver as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.  That is, we follow Section IV A of [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067).

### Throughout this module, we will refer to
* [Pan, Buonanno, Buchman et. al. (2010)](https://arxiv.org/abs/0912.3466v2) as PBB2010,
* [Buonanno, Chen, and Damour (2006)](https://arxiv.org/abs/gr-qc/0508067) as BCD2006, and
* [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows, matching the "steps" listed in [BCD2006](https://arxiv.org/abs/gr-qc/0508067):

0. [Introduction:](#intro) The Physical System of Interest
1. [Step 1:](#step1) Initial Coordinate Choice
1. [Step 2:](#step2) Compute ${\bf r}$, ${\bf p}_{r}$, ${\bf p}_{\theta}$, and ${\bf p}_{\phi}$
1. [Step 3:](#step3) Rotate $\hat{\bf L} \to {\bf e}_{z}$
1. [Step 4:](#step4) Compute $\dot{\bf r}$
1. [Step 5:](#step5) Invert the rotation of Step 3
1. [Step 6:](#step6) Invert the rotation of Step 1
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two compact objects (e.g. black holes or neutron stars) with masses $m_{1}$, $m_{2}$ (in solar masses) and spin angular momenta ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ (see [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.69)) describes the dynamics of this system.  We seek initial conditions for nonadiabatic evolutions of such a system, and follow [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Section IV A.

Please note that throughout this notebook we adpot the following conventions:
1. $c = G = 1$ where $c$ is the speed of light in a vacuum and $G$ is Newton's gravitational constant,
1. hatted vectors (e.g. $\hat{\bf L}_{N}$) usually denote scaled or unit vectors, and
1. the initial inclination angle $\iota$ is chosen to be zero.

Choose a spatial coordinate basis $\left\{ {\bf e}_{0}, {\bf e}_{1}, {\bf e}_{2} \right\}$ so that the initial separation vector ${\bf r}$ between the compact objects lies along the ${\bf e}_{0}$-axis.  We start with the following parameters:
1. binary parameters $m_{1}$, $m_{2}$, ${\bf S}_{1}$, and ${\bf S}_{2}$,
1. initial frequency $f$, and
1. SEOB model parameters (constants dependent on $m_{1}$, $m_{2}$).

Our goal is to produce initial dynamical variables
1. ${\bf x} = \left( x, y, z \right)$, and
1. ${\bf p} = \left( p_{x}, p_{y}, p_{z} \right)$.

We include below the physical parameters necessary to compute the initial conditions.  Solving for the initial conditions requires computing $\frac{ \partial H_{\rm real} }{ \partial p_{i} }$ for $i \in \left\{ r, \theta, \phi \right\}$, <font color='red'>which we defer to another module</font>.

<font color='red'>Please note that in [BCD2006](https://arxiv.org/abs/gr-qc/0508067) the initial conditions are solved for given an initial separation; here we use a given initial frequency instead.  The difference is in our approach to solving Equation (4.8).  Our approach also differs from that found in LALSuite's XLALSimIMRSpinEOBInitialConditionsPrec() function (file: LALSimIMRSpinEOBInitialConditionsPrec.c) because we choose our intial coordinate system so that the inclination angle $\iota$ is zero (this simplifies their step one and step five).</font>

Besides the physical parameters, we also need the [Euler–Mascheroni constant](https://en.wikipedia.org/wiki/Euler%E2%80%93Mascheroni_constant) $\gamma$ and the [geomtrized](https://en.wikipedia.org/wiki/Geometrized_unit_system) solar mass $\mathcal{M}_{\odot}$, both hard-coded in LALSuite with the significant digits shown below.  (The following links are directed to the appropriate LALSuite documentation: [$\gamma$](https://lscsoft.docs.ligo.org/lalsuite/lal/group___l_a_l_constants__h.html#gac6af32574ff8acaeeafe8bf422281e98) and [$\mathcal{M}_{\odot}$](https://lscsoft.docs.ligo.org/lalsuite/lal/group___l_a_l_constants__h.html#gab83f8c705dda3fd0bb2d5f2470bb9cdd).)

In [6]:
import sympy as sp
m1,m2,S10,S11,S12,S20,S21,S22,f = sp.symbols("m1 m2 S10 S11 S12 S20 S21 S22 f",real=True)
dHdpx,dHdpy,dHdpz = sp.symbols("dHdpx dHdpy dHdpz",real=True)

gamma = 0.577215664901532860606512090082402431
Msol = 4.925491025543575903411922162094833998e-6

<a id='step1'></a>

# Step 1: Initial Coordinate Choice \[Back to [top](#toc)\]
$$\label{step1}$$

<a id='massterms'></a>

## Mass terms \[Back to [top](#toc)\]
$$\label{massterms}$$

We begin by defining the following useful quantities:

\begin{align*}
    M &= m_{1} + m_{2} \\
    \eta &= \frac{ m_{1} m_{2} }{ M^{2} }
\end{align*}

In [7]:
M = m1 + m2
Msq = M*M
eta = m1*m2/Msq

<a id='spinterms'></a>

## Spin terms \[Back to [top](#toc)\]
$$\label{spinterms}$$

<font color='red'>Can we find a source that says how the basis for the orientation of ${\bf S}_{1}$, ${\bf S}_{2}$ is chosen?  For now I assume it's in the $\hat{\bf r}, \hat{\bf v}, \hat{\bf L}_{N}$-frame, which is suggestd by the fact that LALSuite rotates them using the same rotation matrix as rotates $\hat{\bf r}$, $\hat{\bf v}$, and $\hat{\bf L}_{N}$ if $\iota \not= 0$.</font>  Since we assumed $G = c = 1$, we make the spin angular momenta dimensionless via:

\begin{align*}
    \hat{\bf S}_{1} &= \frac{ 1 }{ M^{2} } {\bf S}_{1} \\
    \hat{\bf S}_{2} &= \frac{ 1 }{ M^{2} } {\bf S}_{2}
\end{align*}

In [8]:
S1hat0 = S10/Msq
S1hat1 = S11/Msq
S1hat2 = S12/Msq
S2hat0 = S20/Msq
S2hat1 = S21/Msq
S2hat2 = S22/Msq

<a id='rhat'></a>

## Normalized Position $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rhat}$$

We assumed that the initial separation vector ${\bf r}$ lies along the ${\bf e}_{0}$-axis, so the normalized initial separation vector $\hat{\bf r}$ is given by

\begin{align*}
    \hat{\bf r}^{0} &= 1 \\
    \hat{\bf r}^{1} &= 0 \\
    \hat{\bf r}^{2} &= 0.
\end{align*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 803--805.

In [9]:
rhat0 = 1.
rhat1 = 0.
rhat2 = 0.

<a id='ln'></a>

## Normalized Orbital Angular Momenutm $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{ln}$$

Since we assume that the initial separation vector ${\bf r}$ between $m_{1}$ and $m_{2}$ lies along the ${\bf e}_{0}$-axis, the initial orbital plane coincides with the ${\bf e}_{0},{\bf e}_{1}$-plane.  Thus the normalized inital orbital angular momentum vector $\hat{\bf L}_{N}$ is given by

\begin{align*}
    \hat{\bf L}_{N}^{0} &= 0 \\
    \hat{\bf L}_{N}^{1} &= 0 \\
    \hat{\bf L}_{N}^{2} &= 1.
\end{align*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 788--790.

In [10]:
LNhat0 = 0.
LNhat1 = 0.
LNhat2 = 1.

<a id='vhat'></a>

## Normalized Velocity $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{vhat}$$

Given normalized orbital angular momentum ($\hat{\bf L}_{N}$) and normalized position ($\hat{\bf r}$), the normalized velocity vector ($\hat{\bf v}$) is given by

\begin{equation*}
    \hat{\bf v} = \frac{ \hat{\bf L}_{N} \times \hat{\bf r} }{ \left\lvert \hat{\bf L}_{N} \times \hat{\bf r} \right\rvert }.
\end{equation*}

Given $\hat{\bf L}_{N} = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}$ and $\hat{\bf r} = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}$ it is clear that $\hat{\bf v} = \begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix}$.

See LALSimIMRSpinEOBInitialConditionsPrec.c Lines 812--814.

In [11]:
vhat0 = 0.
vhat1 = 1.
vhat2 = 0.

## Note \[Back to [top](#toc)\]


Since we began assuming $\iota = 0$, we do not need to rotate $\hat{\bf r}$, $\hat{\bf v}$, $\hat{\bf L}_{N}$, ${\bf S}_{1}$, ${\bf S}_{2}$, $\hat{\bf S}_{1}$, or $\hat{\bf S}_{2}$ as is done at LALSimIMRSpinEOBInitialConditionsPrec.c Lines 840.

<a id='step2'></a>

# Step 2: Compute ${\bf r}$ and ${\bf p}$ in spherical coordinates \[Back to [top](#toc)\]
$$\label{step2}$$

<a id='phitheta'></a>

## $\phi$ and $\theta$ \[Back to [top](#toc)\]
$$\label{phitheta}$$

Because we assumed the initial separation vector ${\bf r}$ lies along the ${\bf e}_{0}$ axis, we have ${\bf r}^{1} = {\bf r}^{2} = 0$.  In spherical coordinates, the separation vector ${\bf r} = \left( {\bf r}^{r}, {\bf r}^{\phi}, {\bf r}^{\theta} \right)$ therefore has components

\begin{align*}
    {\bf r}^{\phi} &= 0 \\
    {\bf r}^{\theta} &= \frac{ \pi }{ 2 }.
\end{align*}

This is also given in [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Equation 4.7.

In [17]:
rphi = 0.
rtheta = 0.

<a id='omega'></a>

## $\omega$ \[Back to [top](#toc)\]
$$\label{omega}$$

<font color='red'>Is there a paper reference for this formula?  It's not quite Newtonian ($\omega = 2 \pi f)$.</font>
 
\begin{equation*}
    \omega = M \mathcal{M}_{\odot} \pi f.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 893.

In [18]:
omega = M*Msol*sp.pi*f

<a id='velocity'></a>

## Initial Velocity $v$ \[Back to [top](#toc)\]
$$\label{velocity}$$

<font color='red'>Is there a paper reference for this formula?</font>

\begin{equation*}
    v = \sqrt[3]{ \omega }.
\end{equation*}

See LALSimIMRSpinEOBInitialConditionsPrec.c Line 894.

In [19]:
v = sp.cbrt(omega)

<a id='roots'></a>

## Root-finding...

$$\label{roots}$$

<font color='red'>**Note: LALSuite scales the following values; see LALSimIMRSpinEOBInitialConditionsPrec.c Line 899.**</font>

We will write components of the momentum vector ${\bf p}$ in spherical coordinates with components ${\bf p}^{r}$, ${\bf p}^{\phi}$, and ${\bf p}^{\theta}$.

<font color='red'>These are initial guesses for the root-finding routine.</font>  That ${\bf p}^{r} = 0$ is from [BCD2006](https://arxiv.org/abs/gr-qc/0508067) Equation 4.9.

\begin{align*}
    {\bf r}^{r} &= \frac{ 1 }{ v^{2} } \\
    {\bf p}^{r} &= 0 \\
    {\bf p}^{\phi} &= v \\
    {\bf p}^{\theta} &= 0.2.
\end{align*}

Find roots of $\frac{ \partial H }{ \partial {\bf r}^{0}}$, $\frac{ \partial H }{ \partial {\bf p}^{1}}$, and $\frac{ \partial H }{ \partial {\bf r}^{1}}$.  Convert the values to spherical values $\frac{ \partial H }{ \partial {\bf r}^{r}}$, $\frac{ \partial H }{ \partial {\bf p}^{\phi}}$, and $\frac{ \partial H }{ \partial {\bf r}^{\theta}}$.

In [ ]:
rr = 1/(v*v)
pr = 0
pphi = v
ptheta = 0.2

<a id='step3'></a>

# Step 3: Rotate $\hat{\bf L} \to {\bf e}_{z}$ \[Back to [top](#toc)\]
$$\label{step3}$$

<a id='normr'></a>

## Normalize ${\bf r}$ \[Back to [top](#toc)\]
$$\label{normr}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Line 1100, we normalize the new position vector ${\bf q}$ that we found in [Step 2](#step2):

\begin{equation*}
    \hat{\bf q} = \frac{ {\bf q} }{ \left\lvert {\bf q} \right\rvert}.
\end{equation*}

<font color='red'></font>

In [ ]:
qnorminv = 1/sp.sqrt(qx*qx + qy*qy + qz*qz)
qhatx = qx*qnorminv
qhaty = qy*qnorminv
qhatz = qz*qnorminv

<a id='normp'></a>

## Normalize ${\bf p}$ \[Back to [top](#toc)\]
$$\label{normp}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Line 1101, we normalize the new position vector ${\bf p}$ that we found in [Step 2](#step2):

\begin{equation*}
    \hat{\bf p} = \frac{ {\bf p} }{ \left\lvert {\bf p} \right\rvert}.
\end{equation*}

In [ ]:
pnorminv = 1/sp.sqrt(px*px + py*py + pz*pz)
phatx = px*pnorminv
phaty = py*pnorminv
phatz = pz*pnorminv

<a id='lprm'></a>

## ${\bf L}^{\prime}$ \[Back to [top](#toc)\]
$$\label{lprm}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Lines 1103--1105, we compute the relativistic angular momentum vector ${\bf L}^{\prime}$:

\begin{equation*}
    {\bf L}^{\prime} = \hat{\bf q} \times \hat{\bf p}.
\end{equation*}

In [ ]:
Lprmx = qhaty*phatz - qhatz*phaty
Lprmy = qhatz*phatx - qhatx*phatz
Lprmz = qhatx*phaty - qhaty*phatx

<a id='lhat'></a>

## $\hat{\bf L}$ \[Back to [top](#toc)\]
$$\label{lhat}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Line 1107, we normalize ${\bf L}^{\prime}$:

\begin{equation*}
    \hat{\bf L} = \frac{ {\bf L}^{\prime} }{ \left\lvert {\bf L}^{\prime} \right\rvert }.
\end{equation*}

In [ ]:
Lprmnorminv = 1/sp.sqrt(Lprmx*Lprmx + Lprmy*Lprmy + Lprmz*Lprmz)
Lhatx = Lprmx*Lprmnorminv
Lhaty = Lprmy*Lprmnorminv
Lhatz = Lprmz*Lprmnorminv

<a id='rotationmatrix3'></a>

## Rotation matrix 3 \[Back to [top](#toc)\]
$$\label{rotationmatrix3}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Line 1112, the rotation matrix from the $\left\{ \hat{\bf r}, {\bf e}_{y}, \hat{\bf L}_{N} \right\}$ frame to the $\left\{ \hat{\bf r}, {\bf e}_{1}, \hat{\bf L} \right\}$ frame is given by

\begin{equation*}
    \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}.
\end{equation*}

<font color='red'>The matrix to invert the rotation is simply the transpose:</font>

\begin{equation*}
    \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf p}^{x} & \hat{\bf L}^{x} \\
        \hat{\bf q}^{y} & \hat{\bf p}^{y} & \hat{\bf L}^{y} \\
        \hat{\bf q}^{z} & \hat{\bf p}^{z} & \hat{\bf L}^{z}\end{bmatrix}.
\end{equation*}

To see that this is indeed the correct matrix inverse, note that by construction $\hat{\bf q}$, $\hat{\bf p}$, and $\hat{\bf L}$ are all unit vectors orthogonal to one another.

In [ ]:
rotatehat00 = qhatx
rotatehat01 = qhaty
rotatehat02 = qhatz
rotatehat10 = phatx
rotatehat11 = phaty
rotatehat12 = phatz
rotatehat20 = Lhatx
rotatehat21 = Lhaty
rotatehat22 = Lhatz

inverthat00 = qhatx
inverthat01 = phatx
inverthat02 = Lhatx
inverthat10 = qhaty
inverthat11 = phaty
inverthat12 = Lhaty
inverthat20 = qhatz
inverthat21 = phatz
inverthat22 = Lhatz

<a id='rotatehatrhat'></a>

## Rotate $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rotateshatrhat}$$

We rotate $\hat{\bf r}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1117.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf r}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf r}^{x} \\ \hat{\bf r}^{y} \\ \hat{\bf r}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
rhatprmx = qhatx*rhatx + qhaty*rhaty + qhatz*rhatz
rhatprmy = phatx*rhatx + phaty*rhaty + phatz*rhatz
rhatprmz = Lhatx*rhatx + Lhaty*rhaty + Lhatz*rhatz

<a id='rotatehatvhat'></a>

## Rotate $\hat{\bf v}$ \[Back to [top](#toc)\]
$$\label{rotateshatvhat}$$

We rotate $\hat{\bf v}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1118.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf v}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf v}^{x} \\ \hat{\bf v}^{y} \\ \hat{\bf v}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
vhatprmx = qhatx*vhatx + qhaty*vhaty + qhatz*vhatz
vhatprmy = phatx*vhatx + phaty*vhaty + phatz*vhatz
vhatprmz = Lhatx*vhatx + Lhaty*vhaty + Lhatz*vhatz

<a id='rotatehatLNhat'></a>

## Rotate $\hat{\bf L}_{N}$ \[Back to [top](#toc)\]
$$\label{rotateshatLNhat}$$

We rotate $\hat{\bf L}_{N}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1119.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf L}_{N}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf L}_{N}^{x} \\ \hat{\bf L}_{N}^{y} \\ \hat{\bf L}_{N}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
LNhatprmx = qhatx*LNhatx + qhaty*LNhaty + qhatz*LNhatz
LNhatprmy = phatx*LNhatx + phaty*LNhaty + phatz*LNhatz
LNhatprmz = Lhatx*LNhatx + Lhaty*LNhaty + Lhatz*LNhatz

<a id='rotatehatS1'></a>

## Rotate ${\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotateshatS1}$$

We rotate ${\bf S}_{1}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1120.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{1}^{x} \\ {\bf S}_{1}^{y} \\ {\bf S}_{1}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
S1prmx = qhatx*S1x + qhaty*S1y + qhatz*S1z
S1prmy = phatx*S1x + phaty*S1y + phatz*S1z
S1prmz = Lhatx*S1x + Lhaty*S1y + Lhatz*S1z

<a id='rotatehatS2'></a>

## Rotate ${\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotateshatS2}$$

We rotate ${\bf S}_{2}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1121.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} {\bf S}_{2}^{x} \\ {\bf S}_{2}^{y} \\ {\bf S}_{2}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
S2prmx = qhatx*S2x + qhaty*S2y + qhatz*S2z
S2prmy = phatx*S2x + phaty*S2y + phatz*S2z
S2prmz = Lhatx*S2x + Lhaty*S2y + Lhatz*S2z

<a id='rotatehatS1hat'></a>

## Rotate $\hat{\bf S}_{1}$ \[Back to [top](#toc)\]
$$\label{rotateshatS1hat}$$

We rotate $\hat{\bf S}_{1}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1122.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{1}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{1}^{x} \\ \hat{\bf S}_{1}^{y} \\ \hat{\bf S}_{1}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
S1hatprmx = qhatx*S1hatx + qhaty*S1haty + qhatz*S1hatz
S1hatprmy = phatx*S1hatx + phaty*S1haty + phatz*S1hatz
S1hatprmz = Lhatx*S1hatx + Lhaty*S1haty + Lhatz*S1hatz

<a id='rotatehatS2hat'></a>

## Rotate $\hat{\bf S}_{2}$ \[Back to [top](#toc)\]
$$\label{rotateshatS2\hat}$$

We rotate $\hat{\bf S}_{2}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1121.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf S}_{2}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf S}_{2}^{x} \\ \hat{\bf S}_{2}^{y} \\ \hat{\bf S}_{2}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
S2hatprmx = qhatx*S2hatx + qhaty*S2haty + qhatz*S2hatz
S2hatprmy = phatx*S2hatx + phaty*S2haty + phatz*S2hatz
S2hatprmz = Lhatx*S2hatx + Lhaty*S2haty + Lhatz*S2hatz

<a id='rotatehatq'></a>

## Rotate ${\bf q}$ \[Back to [top](#toc)\]
$$\label{rotateshatq}$$

We rotate ${\bf q}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1122.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf q}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} {\bf q}^{x} \\ {\bf q}^{y} \\ {\bf q}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
qprmx = qhatx*qx + qhaty*qy + qhatz*qz
qprmy = phatx*qx + phaty*qy + phatz*qz
qprmz = Lhatx*qx + Lhaty*qy + Lhatz*qz

<a id='rotatehatp'></a>

## Rotate ${\bf p}$ \[Back to [top](#toc)\]
$$\label{rotateshatp}$$

We rotate ${\bf p}$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1122.  We'll use primes to denote the rotated vector.

\begin{equation*}
    {\bf p}^{\prime} = \begin{bmatrix} \hat{\bf q}^{x} & \hat{\bf q}^{y} & \hat{\bf q}^{z} \\
        \hat{\bf p}^{x} & \hat{\bf p}^{y} & \hat{\bf p}^{z} \\
        \hat{\bf L}^{x} & \hat{\bf L}^{y} & \hat{\bf L}^{z}\end{bmatrix}
        \begin{bmatrix} {\bf p}^{x} \\ {\bf p}^{y} \\ {\bf p}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

In [ ]:
pprmx = qhatx*px + qhaty*py + qhatz*pz
pprmy = phatx*px + phaty*py + phatz*pz
pprmz = Lhatx*px + Lhaty*py + Lhatz*pz

<a id='step4'></a>

# Step 4: Compute $\dot{\bf r}$ \[Back to [top](#toc)\]
$$\label{step4}$$

<a id='carttosph'></a>

## Convert from Cartesian to Spherical \[Back to [top](#toc)\]
$$\label{carttosph}$$

Converting from Cartesian to spherical coordinates takes place at LALSimIMRSpinEOBInitialConditionsPrec.c Lines 277--282.  In the special case where $\theta = 0$ and $\phi = \frac{ \pi }{ 2 }$, the spherical position vector ${\bf q}_{\rm sph}$ is given by

\begin{align*}
    {\bf q}_{\rm sph}^{0} = {\bf q}^{x} \\
    {\bf q}_{\rm sph}^{1} = \frac{ \pi }{ 2 } \\
    {\bf q}_{\rm sph}^{2} = 0 \\
\end{align*}

and the spherical momentum vector ${\bf p}_{\rm sph}$ is given by

\begin{align*}
    {\bf p}_{\rm sph}^{0} &= {\bf p}^{x} \\
    {\bf p}_{\rm sph}^{1} &= - {\bf q}^{x}*{\bf p}^{z} \\
    {\bf p}_{\rm sph}^{2} &= {\bf q}^{x}*{\bf p}^{y} \\
\end{align*}

<font color='red'></font>

In [ ]:
qsph0 = qprmx
qsph1 = sp.pi/2
qsph2 = 0
psph0 = pprmx
psph1 = -qprmx*pprmz
psph2 = qprmx*pprmy

<a id='secondderiv'></a>

## Second derivatives of $H_{\rm real}$ \[Back to [top](#toc)\]
$$\label{seconderiv}$$

We need to compute $\frac{ \partial^{2} H_{\rm real} }{ \partial r^{2} }$ and $\frac{ \partial^{2} H_{\rm real} }{ \partial r \partial p_{\phi} }$ (in another module).

<font color='red'></font>

In [ ]:
d2Hdr2 = 0
d2Hdrdpphi = 0

<a id='Hderivs'></a>

## Derivatives of $H_{\rm real}$ \[Back to [top](#toc)\]
$$\label{Hderivs}$$

We need to compute $\frac{ \partial H_{\rm real} }{ \partial \xi }$ where $\xi$ is any of the 12 dynamical variables (in another module).

<font color='red'></font>

In [ ]:
dHdr = 0
dHdtheta = 0
dHdphi = 0
dHdpr = 0
dHdptheta = 0
dHdpphi = 0
dHdS1x = 0
dHdS1y = 0
dHdS1z = 0
dHdS2x = 0
dHdS2y = 0
dHdS2z = 0

<a id='normalizedHdpphi'></a>

## Normalize $\frac{ \partial H }{ \partial p_{\phi} }$ \[Back to [top](#toc)\]
$$\label{normalizedHdpphi}$$

For some reason we normalize $\frac{ \partial H }{ p_{\phi} }$ at LALSimIMRSpinEOBInitialConditionsPrec.c Line 1180.

<font color='red'></font>

In [ ]:
dHdpphi /= sp.sqrt(qprmx*qprmx + qprmy*qprmy + qprmz*qprmz)

<a id='dEdr'></a>

## $\frac{ \partial E }{ \partial r }$ \[Back to [top](#toc)\]
$$\label{dEdr}$$

I don't know what this is or means, but we compute $\frac{ \partial E }{ \partial r }$
<font color='red'></font>

In [ ]:
dEdr = -dHdpphi*d2Hdr2/d2Hdrdpphi

<a id='skerr'></a>

## ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{skerr}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equations (5.2), (5.63), and (5.67) we have

\begin{equation*}
    {\bf S}_{\rm Kerr} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation*}

Taking the square of [BB2010](https://arxiv.org/abs/0912.3517) Equation (4.9) <font color='red'>(be careful with the factor or $M$...)</font>,

\begin{equation*}
    a^{2} = \frac{ {\bf S}_{Kerr} \cdot {\bf S}_{Kerr} }{ M^{2} }
\end{equation*}

so that

\begin{equation*}
    a = \sqrt{ a^{2} }.
\end{equation*}

In [ ]:
SKerr1 = S1x + S2x
SKerr2 = S1y + S2y
SKerr3 = S1z + S2z
asq = (SKerr1*SKerr1 + SKerr2*SKerr2 + SKerr3*SKerr3)/(M*M)
a = sp.sqrt(asq)

<a id='sigmastar'></a>

## $\boldsymbol{\sigma}^{*}$ \[Back to [top](#toc)\]
$$\label{sigmastar}$$

From [BB2010](https://arxiv.org/abs/0912.3517) Equation (5.3),

\begin{equation*}
    \boldsymbol{\sigma}^{*} = \frac{ m_{2} }{ m_{1} } {\bf S}_{1} + \frac{ m_{1} }{ m_{2} }{\bf S}_{2}.
\end{equation*}

In [ ]:
sigmastar1 = S1x*m2/m1 + S2x*m1/m2
sigmastar2 = S1y*m2/m1 + S2y*m1/m2
sigmastar3 = S1z*m2/m1 + S2z*m1/m2

<a id='Hreal'></a>

## $H_{\rm real}$ \[Back to [top](#toc)\]
$$\label{Hreal}$$

We now compute $H_{\rm real}$ (LALSimIMRSpinEOBInitialConditionsPrec.c Line 1222) (another module)

In [ ]:
Hreal = 0

<a id='polardata'></a>

## Polar data \[Back to [top](#toc)\]
$$\label{polardata}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Lines 1234--1238, we set the following polar data $P$ <font color='red'>(need to find reference for this?)</font>:

\begin{align*}
    P^{0} &= {\bf q}_{\rm sph}^{0} \\
    P^{1} &= 0 \\
    P^{2} &= {\bf p}_{\rm sph}^{0} \\
    P^{3} &= {\bf p}_{\rm sph}^{2}
\end{align*}

In [ ]:
polar0 = qsph0
polar1 = 0
polar2 = psph0
polar3 = psph2

<a id='flux'></a>

## Compute Flux \[Back to [top](#toc)\]
$$\label{flux}$$

At LALSimIMRSpinEOBInitialConditionsPrec.c Lines 1249, we copmute flux (another module?)

\begin{align*}
    P^{0} &= {\bf q}_{\rm sph}^{0} \\
    P^{1} &= 0 \\
    P^{2} &= {\bf p}_{\rm sph}^{0} \\
    P^{3} &= {\bf p}_{\rm sph}^{2}
\end{align*}

<a id='step5'></a>

# Step 5: Invert the rotation of Step 3 \[Back to [top](#toc)\]
$$\label{step5}$$

<a id='invrotationmatrix1'></a>

## Inverse Rotation Matrix 1\[Back to [top](#toc)\]
$$\label{invrotationmatrix1}$$

The matrix to invert rotation matrix 1 (see [this cell](#rotationmatrix1)) is simply the transpose:</font>

\begin{equation*}
    \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf v}^{x} & \hat{\bf L}_{N}^{x} \\
        \hat{\bf r}^{y} & \hat{\bf v}^{y} & \hat{\bf L}_{N}^{y} \\
        \hat{\bf r}^{z} & \hat{\bf v}^{z} & \hat{\bf L}_{N}^{z}\end{bmatrix}.
\end{equation*}

To see that this is indeed the correct matrix inverse, note that by construction $\hat{\bf r}$ and $\hat{\bf L}_{N}$ are unit vectors orthogonal to one another.  Furthermore, we noramlized ${\bf v}$ to create $\hat{\bf v}$, and built $\hat{\bf v}$ as a cross produce of $\hat{\bf r}$ and $\hat{\bf L}_{N}$.

In [ ]:
invert00 = rhatx
invert01 = vhatx
invert02 = LNhatx
invert10 = rhaty
invert11 = vhaty
invert12 = LNhaty
invert20 = rhatz
invert21 = vhatz
invert22 = LNhatz

<a id='invrotaterhat'></a>

## Rotate $\hat{\bf r}$ \[Back to [top](#toc)\]
$$\label{rotaterhat}$$

We rotate $\hat{\bf r}$ in LALSimIMRSpinEOBInitialConditionsPrec.c Line 843.  We'll use primes to denote the rotated vector.

\begin{equation*}
    \hat{\bf r}^{\prime} = \begin{bmatrix} \hat{\bf r}^{x} & \hat{\bf r}^{y} & \hat{\bf r}^{z} \\
        \hat{\bf v}^{x} & \hat{\bf v}^{y} & \hat{\bf v}^{z} \\
        \hat{\bf L}_{N}^{x} & \hat{\bf L}_{N}^{y} & \hat{\bf L}_{N}^{z}\end{bmatrix}
        \begin{bmatrix} \hat{\bf r}^{x} \\ \hat{\bf r}^{y} \\ \hat{\bf r}^{z} \end{bmatrix}
\end{equation*}

<font color='red'></font>

rhatprmx = rhatx*rhatx + rhaty*rhaty + rhatz*rhatz
rhatprmy = vhatx*rhatx + vhaty*rhaty + vhatz*rhatz
rhatprnz = LNhatx*rhatx + LNhaty*rhaty + LNhatz*rhatz

<a id='step6'></a>

# Step 6: Invert the rotation of Step 1 \[Back to [top](#toc)\]
$$\label{step6}$$

<a id='tortoise_matrix'></a>

# Tortoise Conversion Matrix \[Back to [top](#toc)\]
$$\label{tortoise_matrix}$$

<font color='red'>We're now back to LALSpinPrecHcapRvecDerivative_v3opt.c, Lines 92--96.</font>

From [Pan, Buonanno, Buchman, et. al. (2010)](https://arxiv.org/abs/0912.3466v2) Equation (A3) the matrix for the coordinate conversion to tortoise coordinates is

\begin{align*}
    \begin{pmatrix} 1 + \frac{ x^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & \frac{ x z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x y }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ y^{2} }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) \\
        \frac{ x z }{ r^{2} } \left( \xi - 1 \right) & \frac{ y z }{ r^{2} } \left( \xi - 1 \right) & 1 + \frac{ z^{2} }{ r^{2} } \left( \xi - 1 \right) \end{pmatrix}
\end{align*}

In [ ]:
ximinus1 = xi - 1
toTort = sp.Array([[1 + x*x*ximinus1/(r*r), x*y*ximinus1/(r*r), x*z*ximinus1/(r*r)],
                   [x*y*ximinus1/(r*r), 1 + y*y*ximinus1/(r*r), y*z*ximinus1/(r*r)],
                   [x*z*ximinus1/(r*r), y*z*ximinus1/(r*r), 1 + z*z*ximinus1/(r*r)]])

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Initial_Conditions.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Initial_Conditions.tex
!rm -f Tut*.out Tut*.aux Tut*.log